# Defining a Baseline Model:
<br>
<b>
Defined a simple model, which is will be used as a comparison with the more advanced models that we will be test. The evaluation and seletion of this model was done in the Baseline Model Experiment-Log.
</b> 

## Loading the Data set:

In [1]:
import pandas as pd
import numpy as np

<b>
Created a pandas dataframe and removed all the NULL values from the dataset.<br>
Replaced the names of Fliars with corresponding integers.<br>
Displayed the head and shape of the data for debugging.
</b>

In [2]:
data = pd.read_csv('f_300_cleaned.csv')
data.fillna("",inplace = True)

#data.replace({'flair': {"AskIndia": 0, "Non-Political": 1, "Scheduled": 2, "Photography": 3,"Science/Technology": 4,
#        "Politics": 5, "Business/Finance": 6, "Policy/Economy": 7,"Sports": 8, "Food": 9 }}, inplace=True)


title_body = data["title"] + ". " + data["body"]  
data = data.assign(title_body = title_body)

Y = data.flair
X = data.combined_features

print(Y.head())
print(Y.shape)

0    Photography
1    Photography
2    Photography
3    Photography
4    Photography
Name: flair, dtype: object
(2422,)


In [3]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold
from sklearn import model_selection

from sklearn import metrics

import pickle

<b>
    Split the data into a training and testing set to evaluate the accuracy and to evaluate if the model generalizes well or if it overfits.
</b>

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 42)

<b>
Created a pipeline which first converts the data into a count matrix followed by its tranformation into a normalized tf or tf-idf representation, this helps in document classification.
The data is then passes to the Linear SVM model.
<br>I have made certain adjustment like, increase the maximum iterations, to the model to suit my data as it could converge properly.
</b>

In [5]:
sgd = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), 
    ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None))])

<b> Here I fit the train data to our pipline.<br>
Further made predicitons on the Test data for determination of accuracy and validation.
</b>

In [6]:
flairs = ["AskIndia", "Non-Political", "Scheduled", "Photography", "Science/Technology",
          "Politics", "Business/Finance", "Policy/Economy","Sports", "Food"]

sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(Y_pred, Y_test))
print(classification_report(Y_test, Y_pred,target_names=flairs))

accuracy 0.6781292984869326
                    precision    recall  f1-score   support

          AskIndia       0.59      0.45      0.51        75
     Non-Political       0.55      0.54      0.55        70
         Scheduled       0.76      0.77      0.77        74
       Photography       0.44      0.18      0.26        65
Science/Technology       0.70      0.96      0.81        77
          Politics       0.54      0.53      0.53        70
  Business/Finance       0.59      0.87      0.70        82
    Policy/Economy       0.88      0.97      0.93        71
            Sports       0.76      0.56      0.64        84
              Food       0.87      0.92      0.89        59

          accuracy                           0.68       727
         macro avg       0.67      0.68      0.66       727
      weighted avg       0.67      0.68      0.66       727



<b>
Pickled the fit model for using it in the WebApp.
</b>

In [7]:
filename = 'SVM.pkl'
pickle.dump(sgd, open(filename, 'wb'))

<b>
Performed K Fold Cross Validation on the model.
    </b>

In [8]:
model = pickle.load(open('SVM.pkl','rb'))

kfold = model_selection.KFold(n_splits=50)
results_kfold = model_selection.cross_val_score(model, X, Y, cv=kfold)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0)) 

Accuracy: 60.33%


'cv_r2_scores_rf = cross_val_score(sgd, X, Y, cv=5,scoring=\'r2\')\nprint(cv_r2_scores_rf)\nprint("Mean 5-Fold R Squared: {}".format(np.mean(cv_r2_scores_rf)))'

<b>Random Testing</b>

In [9]:
model = pickle.load(open('SVM.pkl','rb'))

print(model.predict(["RBI cuts pay check rates"]))
print(model.predict(["Sachin plays cricket with a bat"]))

['Business/Finance']
['Sports']
